In [3]:
# Don't change this part.
# For load graph information and show map
import folium
import pickle


In [2]:
# Part 1~4 and 6
# You can write your code in this file directly.
# Or you can wirte in new .py files and import it.
# ex: from astar import astar
from route_search import bfs, dfs, ucs, astar, astar_time

In [3]:
# Part 5
# Change start ID and end ID.
start = 2270143902; end = 1079387396
#start = 426882161; end = 1737223506
#start = 1718165260; end = 8513026827

In [ ]:
# Don't change this part.
# Show the result of BFS
bfs_path, bfs_dist, bfs_visited = bfs(start, end)
print(f'The number of nodes in the path found by BFS: {len(bfs_path)}')
print(f'Total distance of path found by BFS: {bfs_dist} m')
print(f'The number of visited nodes in BFS: {bfs_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(bfs_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='bfs', weight=4, color='blue'))
fmap

In [ ]:
# Don't change this part.
# Show the result of DFS
dfs_path, dfs_dist, dfs_visited = dfs(start, end)
print(f'The number of nodes in the path found by DFS: {len(dfs_path)}')
print(f'Total distance of path found by DFS: {dfs_dist} m')
print(f'The number of visited nodes in DFS: {dfs_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(dfs_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='dfs', weight=4, color='green'))
fmap

In [ ]:
# Don't change this part.
# Show the result of UCS
ucs_path, ucs_dist, ucs_visited = ucs(start, end)
print(f'The number of nodes in the path found by UCS: {len(ucs_path)}')
print(f'Total distance of path found by UCS: {ucs_dist} m')
print(f'The number of visited nodes in UCS: {ucs_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(ucs_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='ucs', weight=4, color='violet'))
fmap

In [4]:
# Don't change this part.
# Show the result of A* search
import folium
import pickle
def load_path_graph(path):
    with open('graph.pkl', 'rb') as f:
        graph = pickle.load(f)

    node_pairs = list(zip(path[:-1], path[1:]))
    lines = []
    for edge in graph:
        if (edge['u'], edge['v']) in node_pairs or  (edge['v'], edge['u']) in node_pairs:
            lines.append(edge['geometry'])
    return lines
from route_search import bfs, dfs, ucs, astar, astar_time
start = 2270143902; end = 1079387396

import time
st = time.time()
astar_path, astar_dist, astar_visited = astar(start, end)
en = time.time()

print(f'The number of nodes in the path found by A* search: {len(astar_path)}')
print(f'Total distance of path found by A* search: {astar_dist} m')
print(f'The number of visited nodes in A* search: {astar_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(astar_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='astar', weight=4, color='red'))
fmap

print("Execution time",en - st)

 ===== edges table =====
                        end  distance  speed limit
start                                            
2.605931e+07  3.256115e+09    53.141         50.0
2.605931e+07  4.429333e+09    11.548         46.4
2.605931e+07  4.419129e+09    14.588         50.0
2.605931e+07  3.256115e+09    33.469         50.0
2.605931e+07  4.205471e+08    52.411         50.0
...                    ...       ...          ...
8.513027e+09  1.072288e+09    43.595         59.3
8.513027e+09  7.537286e+09    32.068         59.3
8.513027e+09  2.399106e+09    58.160         46.4
8.513027e+09  2.399106e+09     4.288         46.4
8.540464e+09  4.413409e+09    42.700         59.3

[23654 rows x 3 columns]
 ===== nodes table =====
               visited  previous
node                           
2.605931e+07        0         0
2.605931e+07        0         0
2.605932e+07        0         0
2.605932e+07        0         0
2.605932e+07        0         0
...               ...       ...
3.197233e+09    

In [2]:
# Part 6 (Bonus)
# Don't change this part.
# Show the shortest time result of A* search
time_path, time, time_visited = astar_time(start, end)
print(f'The number of nodes in the path found by A* search: {len(time_path)}')
print(f'Total second of path found by A* search: {time} s')
print(f'The number of visited nodes in A* search: {time_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(time_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='astar', weight=4, color='red'))
fmap

The number of nodes in the path found by A* search: 89
Total second of path found by A* search: 320.87823163083164 s
The number of visited nodes in A* search: 789

